In [1]:
import numpy as np
import site, pprint, math, re, numpy
from time import time
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA, KernelPCA
from sklearn.linear_model import RidgeCV
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from lxdr import LXDR
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity as cd, euclidean_distances as ed
import shap

In [2]:
from sklearn.datasets import fetch_openml
df = fetch_openml(data_id='204')
#https://www.openml.org/search?type=data&status=active&id=204

In [3]:
df.data["sex"] = df.data["sex"].astype('float')
df.data["cp"] = df.data["cp"].astype('float')
df.data["fbs"] = df.data["fbs"].astype('float')
df.data["restecg"] = df.data["restecg"].astype('float')
df.data["exang"] = df.data["exang"].astype('float')
df.data["slope"] = df.data["slope"].astype('float')
df.data["thal"] = df.data["thal"].replace(float(np.nan), np.nan).astype('float')

X = df.data.values
y = df.target.values
feature_names = df.feature_names
target_name = df.target_names


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

We will MinMaxScale our data! We will scale them to 0.1, 1, as 0 woyld nullify the importance of a few features.

In [4]:
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
scaler = MinMaxScaler((0.1,1))

In [5]:
X_train_o, X_test_o, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
imp.fit(X_train_o)
X_train_o = imp.transform(X_train_o)
X_test_o = imp.transform(X_test_o)

In [7]:
scaler.fit(X_train_o)
X_train = scaler.transform(X_train_o)
X_test = scaler.transform(X_test_o)

In [8]:
kpca = KernelPCA(3, kernel='rbf', gamma=0.01, random_state=42)
kpca.fit(X_train)
kpca_train = kpca.transform(X_train)
kpca_test  = kpca.transform(X_test)

In [9]:
clf_k = RidgeCV(alphas=(0.01, 100.0, 1.0)).fit(kpca_train, y_train)

And we check the performance, which is slighlty better than the original!

In [10]:
mae_k = mean_absolute_error(y_test, clf_k.predict(kpca_test))
rmse_k  = mean_squared_error(y_test, clf_k.predict(kpca_test), squared=False)
print('KPCA:', mae_k, rmse_k)

KPCA: 41.909430886377535 60.740649999349934


Now, we will test the faithfulness and truthfulness scores when using SHAP.

In [11]:
def shap_predict(instance):
    kpca_instance = kpca.transform(instance)
    return clf_k.predict(kpca_instance)

With the default shap parameters

In [16]:
pred = clf_k.predict(kpca_test)
avg_faith = []
avg_truth = []
avg_instance_diff = []

explanation = shap.KernelExplainer(shap_predict, X_train)
for sample in tqdm.tqdm(range(len(X_test))):
    a = explanation.shap_values(np.array(X_test[sample]))

    #Faith:
    ma = np.aargmax(a)
    temp_sample = X_test[sample].copy()
    if temp_sample[ma] > 0:
        temp_sample[ma] = 0
        w = 1
    elif temp_sample[ma] < 0:
        temp_sample[ma] = 0
        w = -1
    elif temp_sample[ma] == 0:
        temp_sample[ma] = 1
        w = -1
    kpca_sample = kpca.transform([temp_sample])
    temp_pred = clf_k.predict(kpca_sample)[0]
    if a[ma] > 0:
        avg_faith.append(w*(pred[sample]-temp_pred))
    elif a[ma] < 0:
        avg_faith.append(w*(temp_pred-pred[sample]))
    else:
        avg_faith.append((-1)*abs(w*(temp_pred-pred[sample])))       

    #Truth:
    truth = []
    for k in range(len(X_test[sample])):
        temp_sample = X_test[sample].copy()
        if temp_sample[k] > 0:
            temp_sample[k] = 0
            w = 1
        elif temp_sample[k] < 0:
            temp_sample[k] = 0
            w = -1
        elif temp_sample[k] == 0:
            temp_sample[k] = 1
            w = -1
        kpca_sample = kpca.transform([temp_sample])
        temp_pred = clf_k.predict(kpca_sample)[0]
        if a[k] > 0 and w*(pred[sample]-temp_pred)>0:
            truth.append(1)
        elif a[k] < 0 and w*(pred[sample]-temp_pred)<0:
            truth.append(1)
        elif a[k] == 0 and pred[sample]==temp_pred:
            truth.append(1)

    avg_truth.append(np.array(truth).sum()/len(X_test[sample]))
print(round(np.array(avg_faith).mean(),4), round(np.array(avg_instance_diff).mean(),4), round(np.array(avg_truth).mean(),4))

Using 242 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.
100%|██████████| 61/61 [04:45<00:00,  4.69s/it]

7.7533 nan 0.4729



Mean of empty slice.
invalid value encountered in double_scalars
